In [1]:
ls


 Volume in drive C is OS
 Volume Serial Number is 8268-0CB7

 Directory of C:\Users\shrut\Downloads\capstone\final files

05/16/2024  02:44 PM    <DIR>          .
05/16/2024  02:43 PM    <DIR>          ..
05/16/2024  02:44 PM    <DIR>          .ipynb_checkpoints
05/16/2024  02:42 PM            96,382 ab.csv
05/16/2024  02:42 PM            20,040 FInal_Heart.ipynb
05/16/2024  02:42 PM             6,191 logistic_regression_model.joblib
05/16/2024  02:42 PM            14,260 preprocessing_pipeline.joblib
05/16/2024  02:42 PM             2,786 stream.py
               5 File(s)        139,659 bytes
               3 Dir(s)  360,319,291,392 bytes free


In [43]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
df=pd.read_csv("ab.csv",index_col=0)

In [4]:
df

,Recording locations:,Age,Sex,Height,Weight,Pregnancy status,Murmur,Murmur locations,Most audible location,Systolic murmur timing,Systolic murmur grading,Systolic murmur pitch,Diastolic murmur grading,Diastolic murmur pitch,Outcome
0,AV+PV+TV+MV,Child,Female,98.000000,15.900000,False,Absent,MV+PV+TV,AV,0,I/VI,Medium,I/IV,Low,Abnormal
1,AV+PV+TV+MV,Child,Female,103.000000,13.100000,False,Present,AV+MV+PV+TV,TV,Holosystolic,III/VI,High,I/IV,Low,Abnormal
2,AV+PV+TV+MV,Child,Male,115.000000,19.100000,False,Unknown,MV+PV+TV,AV,2,I/VI,Medium,I/IV,Low,Abnormal
3,AV+PV+TV+MV,Child,Male,98.000000,15.900000,False,Present,TV,TV,Holosystolic,I/VI,Low,I/IV,High,Abnormal
4,AV+PV+TV+MV,Child,Male,87.000000,11.200000,False,Present,AV+MV+PV+TV,PV,Early-systolic,II/VI,Low,I/IV,Low,Abnormal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,AV+PV+TV+MV,Child,Male,105.000000,16.600000,False,Absent,MV+PV+TV,AV,937,III/VI,Medium,III/IV,Medium,Normal
938,AV+PV+TV+MV,Child,Male,92.000000,15.200000,False,Absent,MV+PV+TV,MV,938,III/VI,High,III/IV,Medium,Normal
939,AV+PV+TV+MV,Child,Female,97.000000,13.500000,False,Present,MV+TV,TV,Early-systolic,I/VI,Low,III/IV,Low,Abnormal
940,AV+PV,Child,Female,132.000000,38.100000,False,Absent,MV+PV+TV,MV,940,III/VI,Medium,III/IV,Low,Normal


 <b><h1>PRE-PROCESSING</b></h1>

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

# Assuming df is your DataFrame

# Define preprocessing steps
categorical_cols = ['Recording locations:', 'Age', 'Sex', 'Murmur locations', 'Pregnancy status', 'Diastolic murmur grading', 'Systolic murmur grading', 'Murmur', 'Most audible location', 'Systolic murmur timing', 'Systolic murmur pitch', 'Diastolic murmur pitch']
numerical_cols = ['Height', 'Weight']

preprocessing_steps = [
    ('onehot_encoder', OneHotEncoder(drop='first', sparse_output=False)),
    ('scaler', StandardScaler())
]


# Create the column transformer
column_transformer = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_cols),
        ('scaler', StandardScaler(), numerical_cols)
    ],
    remainder='drop'  # <--- Add this parameter
)

# Create the pipeline
preprocessing_pipeline = Pipeline([
    ('column_transformer', column_transformer)
])


In [7]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns="Outcome"), df["Outcome"], test_size=0.3, random_state=42)

# Apply preprocessing pipeline to training data
X_train_transformed = preprocessing_pipeline.fit_transform(X_train)

# Apply preprocessing pipeline to testing data
X_test_transformed = preprocessing_pipeline.transform(X_test)

C:\Users\shrut\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 1, 9] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


# LOGISTIC REGRESSION 

In [8]:
# Train logistic regression model
logistic_model = LogisticRegression()
logistic_model.fit(X_train_transformed, y_train)

# Save the model using joblib
joblib.dump(logistic_model, 'logistic_regression_model.joblib')
joblib.dump(preprocessing_pipeline, 'preprocessing_pipeline.joblib')
y_pred = logistic_model.predict(preprocessing_pipeline.transform(X_test))

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6855123674911661


C:\Users\shrut\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 1, 9] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


## SVM  

In [10]:
from sklearn.svm import SVC

# Train SVM model
svm_model = SVC()
svm_model.fit(X_train_transformed, y_train)

# Save the model and preprocessing pipeline using joblib
joblib.dump(svm_model, 'svm_model.joblib')
joblib.dump(preprocessing_pipeline, 'preprocessing_pipeline.joblib')

# Transform the test data and make predictions
X_test_transformed = preprocessing_pipeline.transform(X_test)
y_pred = svm_model.predict(X_test_transformed)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6537102473498233


C:\Users\shrut\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [0, 1, 9] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


## RANDOM FOREST CLASSIFIER

In [39]:

# Initialize the Random Forest Classifier
model = RandomForestClassifier()

# Train the model
model.fit(X_train_transformed, y_train)

# Save the model
joblib.dump(model, 'random_forest_model.joblib')
joblib.dump(preprocessing_pipeline, 'preprocessing_pipeline.joblib')


# Make predictions on the test set
y_pred = model.predict(X_test_transformed)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.6537102473498233


## GRADIENT BOOSTING CLASSIFIER

In [40]:

# Initialize the Gradient Boosting Classifier
model = GradientBoostingClassifier()

# Train the model
model.fit(X_train_transformed, y_train)

# Save the model
joblib.dump(model, 'gradient_boosting_model.joblib')
joblib.dump(preprocessing_pipeline, 'preprocessing_pipeline.joblib')

# Make predictions on the test set
y_pred = model.predict(X_test_transformed)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.6466431095406361


## MLP CLASSIFIER

In [38]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import joblib

# Initialize the MLP Classifier
model = MLPClassifier()

# Train the model
model.fit(X_train_transformed, y_train)

# Save the model
joblib.dump(model, 'mlp_classifier_model.joblib')
joblib.dump(preprocessing_pipeline, 'preprocessing_pipeline.joblib')


# Make predictions on the test set
y_pred = model.predict(X_test_transformed)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.6148409893992933


C:\Users\shrut\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


## ADABOOST CLASSIFIER

In [37]:

# Initialize the AdaBoost Classifier
model = AdaBoostClassifier()

# Train the model
model.fit(X_train_transformed, y_train)

# Save the model
joblib.dump(model, 'adaboost_model.joblib')
joblib.dump(preprocessing_pipeline, 'preprocessing_pipeline.joblib')

# Make predictions on the test set
y_pred = model.predict(X_test_transformed)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


C:\Users\shrut\anaconda3\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.6890459363957597


## KNN CLASSIFIER

In [36]:
# Initialize the K-Nearest Neighbors Classifier
model = KNeighborsClassifier()

# Train the model
model.fit(X_train_transformed, y_train)

# Save the model
joblib.dump(model, 'knn_model.joblib')
joblib.dump(preprocessing_pipeline, 'preprocessing_pipeline.joblib')


# Make predictions on the test set
y_pred = model.predict(X_test_transformed)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6219081272084805


## LIGHTGBM CLASSIFIER

In [35]:
# Initialize the LightGBM Classifier
model = LGBMClassifier()

# Train the model
model.fit(X_train_transformed, y_train)

# Save the model
joblib.dump(model, 'lightgbm_model.joblib')
joblib.dump(preprocessing_pipeline, 'preprocessing_pipeline.joblib')

# Make predictions on the test set
y_pred = model.predict(X_test_transformed)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

[LightGBM] [Info] Number of positive: 348, number of negative: 311
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000184 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 378
[LightGBM] [Info] Number of data points in the train set: 659, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.528073 -> initscore=0.112410
[LightGBM] [Info] Start training from score 0.112410
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

## GAUSSIAN PROCESS CLASSIFIER

In [34]:
# Initialize the Gaussian Process Classifier with the defined kernel
model = GaussianProcessClassifier(kernel=kernel)

# Train the model
model.fit(X_train_transformed, y_train)

# Save the model
joblib.dump(model, 'gaussian_process_model.joblib')
joblib.dump(preprocessing_pipeline, 'preprocessing_pipeline.joblib')

# Make predictions on the test set
y_pred = model.predict(X_test_transformed)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.696113074204947
